# Combining & Exporting Data - Part 2

## Content
#### 1. Import libraries and data
#### 2. Merging data
#### 3. Exporting data

# 1. Import libraries and data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create project folder path
path = r'C:\Users\Lara\Career Foundry Projects\21-09-2023 Instacart Basket Analysis'

In [3]:
# Import datasets products_checked.csv and orders_producs_combined.pkl
df_prods = pd.read_csv (os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'), index_col = False)
df_op_comb = pd.read_pickle (os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))

In [4]:
# Check output and shape for both df-s
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [5]:
df_prods.shape

(49693, 5)

In [6]:
df_op_comb.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


In [7]:
df_op_comb.shape

(32434489, 10)

# 2. Merging data

#### Most suitable way to combine df_op_comb and df_prods would be through merging with inner join to avoid a lot of missing values and using column 'product_id' as key and using merge flag to see if merge was full
#### Column _merge in df_op_comb, representing flag from previous merge of another 2 df-s, will not be necessery in upcoming merge

In [9]:
# Drop column _merge from df_op_comb
df_op_comb_1 = df_op_comb.drop(columns = ['_merge'])

In [10]:
# Check output
df_op_comb_1.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,196,1,0
1,2539329,1,1,2,8,NaN,14084,2,0
2,2539329,1,1,2,8,NaN,12427,3,0
3,2539329,1,1,2,8,NaN,26088,4,0
4,2539329,1,1,2,8,NaN,26405,5,0


In [12]:
# Merge df_op_comb_1 and df_prods with column 'product_id' as key and with merge flag
df_op_merged = df_op_comb_1.merge(df_prods, on = 'product_id', indicator = True)

In [13]:
# Check output
df_op_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both


In [20]:
# Check shape
df_op_merged.shape

(32434212, 14)

In [14]:
# Check if it's full merge
df_op_merged['_merge'].value_counts()

_merge
both          32434212
left_only            0
right_only           0
Name: count, dtype: int64

In [15]:
# Outer merge
df_op_merged_out = df_op_comb_1.merge(df_prods, on = 'product_id', indicator = True, how = 'outer')

In [16]:
# Check if it's full merge
df_op_merged_out['_merge'].value_counts()

_merge
both          32434212
left_only         2029
right_only          11
Name: count, dtype: int64

#### df_op_merged does not represent full merge, but as it turns out all together 2029 + 11 = 2040 rows that are flaged with  left_only and right_only can be omitted (number is very little compared to 32 millions)
#### Merge flag is needed through merging process only and we can drop this column before exporting data.

In [18]:
# Drop merge flag
df_op_merged_1 = df_op_merged.drop(columns = ['_merge'])

In [19]:
# Check shape
df_op_merged_1.shape

(32434212, 13)

# 3. Exporting data

#### Since df created through merging process has 32 million rows, data should be exported as pikle format

In [21]:
# Exporting df_op_merged_1
df_op_merged_1.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))